<a href="https://colab.research.google.com/github/RuchiRaina3/Baby-Cry-Project/blob/main/Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Classification is done using Sequential Neural Network
It has five layers. First four layers have relu activation function and fifth layer has the softmax activation function.

### Features extracted are MFCC. 
Mel-frequency cepstral coefficients (MFCCs) are coefficients that collectively make up an MFC. For that I have used librosa.feature.mfcc(). It will return 125 MFCCs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from numpy import array
import librosa

import sys, time, os, warnings
warnings.filterwarnings("ignore")
import re

import keras
from tqdm import tqdm

from keras.models import Sequential

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, BatchNormalization
from keras.layers.merge import add

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
import pandas as pd
# Read the Excel File
data = pd.read_csv("/content/drive/MyDrive/Baby Cry Project/Baby Cry Data.xlsx")

In [ ]:
audio_path=[]
for i in range (len(data)):
  a=data.iloc[i][0]
  audio_path.append(a)
  i=i+1
  # print(a)

label=[]
for i in range (len(data)):
  a=data.iloc[i][1]
  label.append(a)
  i=i+1

In [ ]:
#Function For Feature Extraction
def features_extractor(file):
    #load the file (audio)
    audio, sample_rate = librosa.load(file_name, sr=8000) 
    #we extract mfcc
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=125)
    #in order to find out scaled feature we do mean of transpose of value
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

In [ ]:
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for i in range(len(data)):
  file_name = data.iloc[i][0]
  final_class_labels= data.iloc[i][1] 
  data=features_extractor(file_name)
  extracted_features.append([data,final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])

In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
X.shape

In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
### Label Encoding -> Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
### No of classes
num_labels=y.shape[1]

In [ ]:
model=Sequential()
###first layer
model.add(Dense(400,input_shape=(125,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###fourth layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
# from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
num_epochs = 3000

num_batch_size = 53

# checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/baby_cry_4_sec_audio/checkpoint', 
                              #  verbose=1, save_best_only=True)
start = datetime.now()
# model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=1)
duration = datetime.now() - start
model.save('model')
print("Training completed in time: ", duration)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8352941274642944


In [ ]:
test_accuracy

[1.6455330848693848, 0.8352941274642944]

In [ ]:
y_test

In [ ]:
pred = model.predict(X_test) 
# print(pred)
pred = np.argmax(pred, axis = 1)
label = np.argmax(y_test,axis = 1)


[0 2 0 1 2 2 0 3 4 4 0 3 1 4 4 2 2 4 4 3 0 0 3 0 4 4 4 2 2 2 4 0 3 3 2 4 0
 1 2 4 4 3 1 4 2 2 2 2 4 2 0 4 3 4 2 4 0 0 4 1 4 3 0 2 2 1 2 2 2 1 4 4 0 0
 0 0 4 0 2 4 3 3 3 4 0]
[2 4 0 1 2 2 0 3 2 3 0 2 1 3 4 2 2 4 4 3 0 0 3 0 4 4 3 2 2 2 4 0 3 3 2 4 0
 1 2 4 0 3 0 4 2 2 2 2 3 2 0 4 3 4 2 4 0 0 4 1 3 3 0 2 2 2 2 2 4 4 4 4 0 0
 0 0 4 0 2 4 3 3 3 4 0]


In [ ]:
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0


In [ ]:
accuracy_metric(label,pred)

83.52941176470588